# PEFT库使用
## 1.PeftConfig
- 每一个peft方法都对应一个config

In [ ]:
from peft import PeftConfig, LoraConfig


lora_config = LoraConfig(
    task_type="CAUSAL_LM", #type of task to train model on
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)


## 2.PeftModel
- 可以在transformers中加载任何模型后, 使用`get_peft_model`将其转化为`PeftModel`
- 调用`get_peft_model`后, the base model will be modified in-place.
- 或者使用`Model.add_adapter`添加(多个adapter)

In [ ]:
from peft import get_peft_model, PeftModel
from transformers import AutoModel

ckp = ""
model = AutoModel.from_pretrained(ckp)

peft_model = get_peft_model(
    model=model, 
    peft_config=lora_config
)
# model.add_adapter(lora_config, adapter_name="aaa")
# model.set_adapter("aaa")
peft_model.print_trainable_parameters()


## 2.训练
- 得到`PeftModel`后, 使用`Trainer`或者自己写的Pytorch训练脚本即可训练
- 保存只保存额外的peft权重, 例如用lora训练后只保存`adapter_config.json`和`adapter_model.safetensors`

In [ ]:
from transformers import TrainingArguments, Trainer

train_arg = TrainingArguments()
trainer = Trainer(
    model=peft_model,
    args=train_arg
)
trainer.train()
peft_model.save_pretrained("output_dir")


## 6.推理
- 加载一个peft后的模型, 应注意文件是否有`adapter_config.json`
- By default, the PeftModel is set for inference, but if you’d like to train the adapter some more you can set is_trainable=True
- The PeftModel.from_pretrained() method is the most flexible way to load a PeftModel because it doesn’t matter what model framework was used (Transformers, timm, a generic PyTorch model). Other classes, like AutoPeftModel, are just a convenient wrapper around the base PeftModel

In [ ]:
from peft import AutoPeftModelForCausalLM

save_ckp = ""
inference_model = AutoPeftModelForCausalLM.from_pretrained(save_ckp)
